# Setup a computer with AiiDA 

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent
from aiida.backends.utils import get_automatic_user, get_backend_type

if get_backend_type() == 'sqlalchemy':
    from aiida.backends.sqlalchemy.models.authinfo import DbAuthInfo
else:
    from aiida.backends.djsite.db.models import DbAuthInfo

import ipywidgets as ipw
from IPython.display import display, clear_output

from subprocess import check_call, check_output, call
from os import path
import re
import pexpect
import paramiko

In [ ]:
layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}
proxyhost="lsmosrv1.epfl.ch"

In [ ]:
username = "ubuntu"
computer_name = "workhorse"

## Step 1: Setup ssh

In [ ]:
def is_host_known(clustername):
    fn = path.expanduser("~/.ssh/known_hosts")
    if not path.exists(fn):
        return False
    return call(["ssh-keygen", "-F", clustername]) == 0

def make_host_known(clustername, proxycmd=[]):
    fn = path.expanduser("~/.ssh/known_hosts")
    print("Adding keys from %s to %s"%(clustername, fn))
    hashes = check_output(proxycmd+["ssh-keyscan", "-H", clustername])
    with open(fn, "a") as f:
        f.write(hashes)

def can_login(clustername, username):
    userhost = username+"@"+clustername
    print("Trying ssh "+userhost+"... ", end='')
    ret = call(["ssh", userhost, "true"])
    print("Ok" if ret==0 else "Failed")
    return ret==0

def is_in_config(clustername):
    fn = path.expanduser("~/.ssh/config")
    if not path.exists(fn):
        return False
    cfglines = open(fn).read().split("\n")
    return "Host "+clustername in cfglines

def write_ssh_config(clustername, hostname, username, proxyhost=None):
    fn = path.expanduser("~/.ssh/config")
    print("Adding section to "+fn)
    with open(fn, "a") as f:
        f.write("Host "+clustername+"\n")
        f.write("Hostname "+hostname+"\n")
        f.write("User "+username+"\n")
        if proxyhost is not None:
            f.write("ProxyCommand ssh -q -Y "+username+"@"+proxyhost+" netcat %h %p\n")
        f.write("ServerAliveInterval 5\n")

def fetch_slurm_accounts(hostname, username, clustername):
    import re
    userhost = username+"@"+clustername
    print("Fetching slurm accounts for cluster {}".format(clustername))
    
    accounts_raw = check_output(["ssh", userhost, 
                             "sacctmgr", "show", "associations", "User={}".format(username),
                             "Cluster={}".format(clustername), "Format='Account'", "--noheader",
                             "--parsable"])
                             # "sinfo", "--noheader", "-o", "%P"])
    slurm_accounts = [_.split("|")[0].strip() for _ in accounts_raw.splitlines()]
    options = [("Please select a slurm account", False)]
    for l in slurm_accounts:
        # The second option (here None) is for partition/constraint
        options.append(["%s"%l, [l,None]])
    drop_account.options = options

    
def setup_ssh_no_proxy(targethost, username, clustername):
    ssh_keygen()
    
    # now setup target host
    if not is_host_known(clustername):
        make_host_known(clustet)

    if not is_in_config(targethost):
        write_ssh_config(targethost, username)

    # final check
    if can_login(targethost, username):
        fetch_slurm_accounts(targethost, username, clustername)
        print("Automatic ssh setup successful :-)")
        return True
    else:
        print("Automatic ssh setup failed, sorry :-(")
        return False

In [ ]:
def on_setup_ssh(b):
    with setup_ssh_out:
        clear_output()
        if len(ip_computer.value) == 0:
            print("Please specify the computer")
            return
        setup_ssh_no_proxy(ip_computer.value, inp_username, clustename)


ip_computer = ipw.Text(description="Computer IP", layout=layout, style=style)
btn_setup_ssh = ipw.Button(description="Setup ssh")
btn_setup_ssh.on_click(on_setup_ssh)
setup_ssh_out = ipw.Output()


display(ip_computer,btn_setup_ssh, setup_ssh_out)

## Step 2: Setup AiiDA Computer

In [ ]:
# https://github.com/aiidateam/aiida_core/blob/develop/aiida/cmdline/commands/computer.py#L400
from aiida.transport.plugins.ssh import parse_sshconfig

def configure_computer(computer_name, username):
    # create DbAuthInfo
    authparams = {
        'compress': True,
        'gss_auth': False,
        'gss_deleg_creds': False,
        'gss_host': computer_name,
        'gss_kex': False,
        'key_policy': 'WarningPolicy',
        'load_system_host_keys': True,
        'port': 22,
        'timeout': 60,
        'username': inp_username.value,
    }
    aiidauser = get_automatic_user()
    authinfo = DbAuthInfo(dbcomputer=Computer.get(computer_name).dbcomputer, aiidauser=aiidauser)
    authinfo.set_auth_params(authparams)
    authinfo.save()
    
    ! verdi computer show {computer_name}    



def setup_computer(computer_name, username):
    try:
        computer = Computer.get(computer_name)
        print("A computer called {} already exists.".format(computer_name))
        return
    except NotExistent:
        pass

    print("Creating new computer with name '{}'".format(computer_name))
    computer = Computer(name=computer_name)
    computer.set_hostname(drop_computer.value['host'])
    computer.set_description(drop_computer.value['description'])
    computer.set_enabled_state(True)
    computer.set_transport_type("ssh")
    computer.set_scheduler_type("slurm")
    computer.set_workdir("/scratch/{username}/aiida_run_"+computer_name)
    # set_mpirun_command() must be called after set_scheduler_type()
    #cmd = "srun -n {tot_num_mpiprocs} -c $SLURM_CPUS_PER_TASK --cpu_bind=rank --hint=nomultithread"
    cmd = "srun -n {tot_num_mpiprocs}"
    computer.set_mpirun_command(cmd.split())
    ncpus = 24
    computer.set_default_mpiprocs_per_machine(ncpus)
    # Note: it would be better to use the custom scheduler commands, but they are not supported in Computer,
    # only in calc...
    prepend_text  = "### computer prepend_text start ###\n"
    prepend_text += "### computer prepend_text end ###\n"
    computer.set_prepend_text(prepend_text)
    computer.store()
    
    configure_computer(computer_name, username)

In [ ]:
def on_setup_computer(b):
    with setup_comp_out:
        clear_output()
        setup_computer(computer_name, username)

    
drop_account = ipw.Dropdown(description="Slurm Account:", options={"first run ssh setup":False}, style=style, layout=layout )
btn_setup_comp = ipw.Button(description="Setup Computer")
btn_setup_comp.on_click(on_setup_computer)
setup_comp_out = ipw.Output()
display(drop_account, btn_setup_comp, setup_comp_out)

## Step 3: Test Computer

In [ ]:
def on_test_computer(b):
    with test_out:
        clear_output()
        if len(drop_computer.value['name'].strip())==0:
            print("Please enter computer name")
            return
        ! verdi computer test --traceback {computer_name}

test_out = ipw.Output()
btn_test_comp = ipw.Button(description="Test Computer")
btn_test_comp.on_click(on_test_computer)
display(btn_test_comp, test_out)